In [ ]:
# default_exp vision.models.xresnet

# Flexible XResNet
> Fastai's XResNet, but with more flexibility.

In [ ]:
#export
import torch.nn as nn
from fastcore.basics import store_attr
from fastcore.meta import delegates
from fastai.layers import ConvLayer, MaxPool, AdaptiveAvgPool, Flatten, defaults
from fastai.vision.models.xresnet import init_cnn
from fastxtend.vision.models.resnet_blocks import *
from fastxtend.imports import *

## XResNet Definition-

In [ ]:
#export
class XResNet(nn.Sequential):
    @delegates(ResBlock)
    def __init__(self, block, expansion, layers, p=0.0, c_in=3, n_out=1000, stem_szs=(32,32,64), 
                 block_szs=[64,128,256,512], widen=1.0, sa=False, act_cls=defaults.activation, ndim=2, 
                 ks=3, stride=2, stem_layer=ConvLayer, stem_pool=MaxPool, head_pool=AdaptiveAvgPool, 
                 custom_head=None, **kwargs):
        store_attr('block,expansion,act_cls,ndim,ks')
        if ks % 2 == 0: raise Exception('kernel size has to be odd!')
        stem_szs = [c_in, *stem_szs]
        stem = [stem_layer(stem_szs[i], stem_szs[i+1], ks=ks, stride=stride if i==0 else 1,
                           act_cls=act_cls, ndim=ndim)
                for i in range(3)]

        assert len(layers) == len(block_szs), 'Length of `layers` must match `block_szs`'
        block_szs = [int(o*widen) for o in block_szs]
        block_szs = [stem_szs[-1]//expansion] + block_szs
        stem_pool = stem_pool(ks=ks, stride=stride, padding=ks//2, ndim=ndim)
        if not is_listy(stem_pool): stem_pool = [stem_pool]
        blocks    = self._make_blocks(layers, block_szs, sa, stride, **kwargs)

        if custom_head:
            head = custom_head(block_szs[-1]*expansion, n_out)
            if not is_listy(head): head = [head]
            body = nn.Sequential(*stem, *stem_pool, *blocks)
            init_cnn(body)
            super().__init__(*list(body), *head)
        else:
            head = self._make_head(block_szs[-1]*expansion, head_pool, ndim, p, n_out)
            super().__init__(*stem, *stem_pool, *blocks, *head)
            init_cnn(self)

    def _make_blocks(self, layers, block_szs, sa, stride, **kwargs):
        return [self._make_layer(ni=block_szs[i], nf=block_szs[i+1], blocks=l,
                                 stride=1 if i==0 else stride, sa=sa and i==len(layers)-4, **kwargs)
                for i,l in enumerate(layers)]

    def _make_layer(self, ni, nf, blocks, stride, sa, **kwargs):
        return nn.Sequential(
            *[self.block(self.expansion, ni if i==0 else nf, nf, stride=stride if i==0 else 1,
                      sa=sa and i==(blocks-1), act_cls=self.act_cls, ndim=self.ndim, ks=self.ks, **kwargs)
              for i in range(blocks)])

    def _make_head(self, ni, head_pool, ndim, p, n_out):
        return [head_pool(sz=1, ndim=ndim), Flatten(), nn.Dropout(p), nn.Linear(ni, n_out)]

## XResNet -

In [ ]:
#export
@delegates(XResNet)
def xresnet18(n_out=1000, c_in=3, p=0.0, act_cls=defaults.activation, **kwargs): 
    return XResNet(ResBlock, 1, [2, 2, 2, 2], n_out=n_out, **kwargs)

@delegates(XResNet)
def xresnet34(n_out=1000, **kwargs):  
    return XResNet(ResBlock, 1, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xresnet50(n_out=1000, **kwargs): 
    return XResNet(ResBlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xresnet101(n_out=1000, **kwargs): 
    return XResNet(ResBlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XResNeXt -

In [ ]:
#export
@delegates(XResNet)
def xresnext18(n_out=1000, **kwargs): 
    return XResNet(ResNeXtBlock, 1, [2, 2, 2, 2], n_out=n_out, **kwargs)

@delegates(XResNet)
def xresnext34(n_out=1000, **kwargs):  
    return XResNet(ResNeXtBlock, 1, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xresnext50(n_out=1000, **kwargs): 
    return XResNet(ResNeXtBlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xresnext101(n_out=1000, **kwargs): 
    return XResNet(ResNeXtBlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XSE-ResNet -

In [ ]:
#export
@delegates(XResNet)
def xse_resnet18(n_out=1000, **kwargs):   
    return XResNet(SEBlock, 1, [2, 2, 2, 2], n_out=n_out, **kwargs)

@delegates(XResNet)
def xse_resnet34(n_out=1000, **kwargs):   
    return XResNet(SEBlock, 1, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xse_resnet50(n_out=1000, **kwargs): 
    return XResNet(SEBlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xse_resnet101(n_out=1000, **kwargs):   
    return XResNet(SEBlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XSE-ResNeXt -

In [ ]:
#export
@delegates(XResNet)
def xse_resnext18(n_out=1000, **kwargs):  
    return XResNet(SEResNeXtBlock, 1, [2, 2, 2, 2], n_out=n_out, **kwargs)

@delegates(XResNet)
def xse_resnext34(n_out=1000, **kwargs):  
    return XResNet(SEResNeXtBlock, 1, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xse_resnext50(n_out=1000, **kwargs):  
    return XResNet(SEResNeXtBlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xse_resnext101(n_out=1000, **kwargs): 
    return XResNet(SEResNeXtBlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XECA-ResNet -

In [ ]:
#export
@delegates(XResNet)
def xeca_resnet18(n_out=1000, **kwargs):   
    return XResNet(ECABlock, 1, [2, 2, 2, 2], n_out=n_out, **kwargs)

@delegates(XResNet)
def xeca_resnet34(n_out=1000, **kwargs):   
    return XResNet(ECABlock, 1, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xeca_resnet50(n_out=1000, **kwargs): 
    return XResNet(ECABlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xeca_resnet101(n_out=1000, **kwargs):   
    return XResNet(ECABlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XECA-ResNeXt -

In [ ]:
#export
@delegates(XResNet)
def xeca_resnext18(n_out=1000, **kwargs):  
    return XResNet(ECAResNeXtBlock, 1, [2, 2, 2, 2], n_out=n_out, **kwargs)

@delegates(XResNet)
def xeca_resnext34(n_out=1000, **kwargs):  
    return XResNet(ECAResNeXtBlock, 1, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xeca_resnext50(n_out=1000, **kwargs):  
    return XResNet(ECAResNeXtBlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xeca_resnext101(n_out=1000, **kwargs): 
    return XResNet(ECAResNeXtBlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XSA-ResNet -

In [ ]:
#export
@delegates(XResNet)
def xsa_resnet18(n_out=1000, **kwargs):   
    return XResNet(SABlock, 1, [2, 2, 2, 2], n_out=n_out, sa_grps=32, **kwargs)

@delegates(XResNet)
def xsa_resnet34(n_out=1000, **kwargs):   
    return XResNet(SABlock, 1, [3, 4, 6, 3], n_out=n_out, sa_grps=32, **kwargs)

@delegates(XResNet)
def xsa_resnet50(n_out=1000, **kwargs): 
    return XResNet(SABlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xsa_resnet101(n_out=1000, **kwargs):   
    return XResNet(SABlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XSA-ResNeXt -

In [ ]:
#export
@delegates(XResNet)
def xsa_resnext18(n_out=1000, **kwargs):  
    return XResNet(SAResNeXtBlock, 1, [2, 2, 2, 2], n_out=n_out, sa_grps=32, **kwargs)

@delegates(XResNet)
def xsa_resnext34(n_out=1000, **kwargs):  
    return XResNet(SAResNeXtBlock, 1, [3, 4, 6, 3], n_out=n_out, sa_grps=32, **kwargs)

@delegates(XResNet)
def xsa_resnext50(n_out=1000, **kwargs):  
    return XResNet(SAResNeXtBlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xsa_resnext101(n_out=1000, **kwargs): 
    return XResNet(SAResNeXtBlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XTA-ResNet -

In [ ]:
#export
@delegates(XResNet)
def xta_resnet18(n_out=1000, **kwargs):   
    return XResNet(TABlock, 1, [2, 2, 2, 2], n_out=n_out, **kwargs)

@delegates(XResNet)
def xta_resnet34(n_out=1000, **kwargs):   
    return XResNet(TABlock, 1, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xta_resnet50(n_out=1000, **kwargs): 
    return XResNet(TABlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xta_resnet101(n_out=1000, **kwargs):   
    return XResNet(TABlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)

## XTA-ResNeXt -

In [ ]:
#export
@delegates(XResNet)
def xta_resnext18(n_out=1000, **kwargs):  
    return XResNet(TAResNeXtBlock, 1, [2, 2, 2, 2], n_out=n_out, sa_grps=32, **kwargs)

@delegates(XResNet)
def xta_resnext34(n_out=1000, **kwargs):  
    return XResNet(TAResNeXtBlock, 1, [3, 4, 6, 3], n_out=n_out, sa_grps=32, **kwargs)

@delegates(XResNet)
def xta_resnext50(n_out=1000, **kwargs):  
    return XResNet(TAResNeXtBlock, 4, [3, 4, 6, 3], n_out=n_out, **kwargs)

@delegates(XResNet)
def xta_resnext101(n_out=1000, **kwargs): 
    return XResNet(TAResNeXtBlock, 4, [3, 4, 23, 3], n_out=n_out, **kwargs)